In [1]:
import os
import shutil

# Path to the directory in the dataset
dataset_dir = '/kaggle/input/my45kstable/stable_signature main'

# Destination working directory
working_dir = '/kaggle/working/'

# Create the destination directory if it doesn't exist
if not os.path.exists(working_dir):
    os.makedirs(working_dir)

# Function to copy files and directories recursively
def copy_tree(src, dst):
    for item in os.listdir(src):
        src_path = os.path.join(src, item)
        dst_path = os.path.join(dst, item)
        if os.path.isdir(src_path):
            if not os.path.exists(dst_path):
                os.makedirs(dst_path)
            copy_tree(src_path, dst_path)
        else:
            shutil.copy2(src_path, dst_path)

# Copy all items from dataset directory to working directory
copy_tree(dataset_dir, working_dir)

print(f"All items copied to {working_dir}")

All items copied to /kaggle/working/


In [3]:
!pip install -r requirements.txt

In [4]:
!apt-get update
!apt-get install -y libmagic1

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]                
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]                           
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,311 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [66.7 kB]       
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]             
Get:11 https://r2u.stat.illinois.edu/ubuntu 

In [ ]:
!pip install pytorch-lightning==1.9.0
!pip install xformers==0.0.28
!pip install python-magic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 kB 13.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 40.5 MB/s eta 0:00:00
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.6.0
    Uninstalling torchmetrics-0.6.0:
      Successfully uninstalled torchmetrics-0.6.0
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 2.5.0.post0
    Uninstalling pytorch-lightning-2.5.0.post0:
      Successfully uninstalled pytorch-lightning-2.5.0.post0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 89.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 797.1/797.1 MB 224.3 MB/s eta 0:00:0100:01

In [ ]:
!pip install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121

In [6]:
%%writefile utils_new_mod.py
import torch
import torch.nn as nn
import importlib
import torch.nn.functional as F

class StegNetDecoder(nn.Module):
   def __init__(self):
      super(StegNetDecoder, self).__init__()

      self.decoder_layers1 = nn.Conv2d(3,256,kernel_size=3,padding=1)
      self.decoder_layers2 = nn.Conv2d(256,128,kernel_size=3,padding=1)

      self.decoder_layers3= nn.Conv2d(128,64,kernel_size=3,padding=1)
      self.decoder_layers4= nn.Conv2d(64,64,kernel_size=3,padding=1)

      self.decoder_layers5= nn.Conv2d(64,32,kernel_size=3,padding=1)

      self.decoder_payload1 =nn.Conv2d(32,16,kernel_size=3,padding=1)
      self.decoder_payload2 = nn.Conv2d(16,16,kernel_size=3,padding=1)
      self.decoder_payload3 = nn.Conv2d(16,8 ,kernel_size=3,padding=1)
      self.decoder_payload4 = nn.Conv2d(8,8,kernel_size=3,padding=1)
      self.decoder_payload5 = nn.Conv2d(8,3,kernel_size=3,padding=1)
      self.decoder_payload6 = nn.Conv2d(3,1,kernel_size=3,padding=1)

      self.decoder_source1 = nn.Conv2d(32,16,kernel_size=3,padding=1)
      self.decoder_source2 = nn.Conv2d(16,16,kernel_size=3,padding=1)
      self.decoder_source3 = nn.Conv2d(16,8,kernel_size=3,padding=1)
      self.decoder_source4 = nn.Conv2d(8,8,kernel_size=3,padding=1)
      self.decoder_source5 = nn.Conv2d(8,3,kernel_size=3,padding=1)
      self.decoder_source6 = nn.Conv2d(3,3,kernel_size=3,padding=1)

   def forward(self,img_w):
       
#       img_w = img_w.view((-1,3,32,32))
      img_w = F.relu(self.decoder_layers1(img_w))
      img_w = F.relu(self.decoder_layers2(img_w))

      img_w = F.relu(self.decoder_layers3(img_w))
      img_w = F.relu(self.decoder_layers4(img_w))

      init_d = F.relu(self.decoder_layers5(img_w))

      img_w = F.relu(self.decoder_payload1(init_d))
      img_w = F.relu(self.decoder_payload2(img_w))
      
      img_w = F.relu(self.decoder_payload3(img_w))
      img_w = F.relu(self.decoder_payload4(img_w))

      img_w=F.relu(self.decoder_payload5(img_w))
      decoded_payload = self.decoder_payload6(img_w)

      img_w = F.relu(self.decoder_source1(init_d))
      img_w = F.relu(self.decoder_source2(img_w))

      img_w = F.relu(self.decoder_source3(img_w))
      img_w = F.relu(self.decoder_source4(img_w))

      img_w = F.relu(self.decoder_source5(img_w))
      decoded_source = self.decoder_source6(img_w)

      return decoded_payload


def get_steg_decoder():
   decoder= StegNetDecoder();
   return decoder

def get_decoder_ckpt(ckpt_path):
   ckpt = torch.load(ckpt_path,map_location="cpu")
   decoder_ckpt =  {k: v for k, v in ckpt['model_state_dict'].items() if 'decoder_' in k}
   return decoder_ckpt
   

Writing utils_new_mod.py


In [7]:
!pip install torch-fidelity


In [8]:
%%writefile headers.py
import argparse
import json
import os
import sys
from copy import deepcopy
from omegaconf import OmegaConf
from pathlib import Path
from typing import Callable, Iterable
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
import torchvision
import utils
import utils_img
import utils_model
import torchvision.transforms as T
sys.path.append('src')
from ldm.models.autoencoder import AutoencoderKL
from ldm.models.diffusion.ddpm import LatentDiffusion
from loss.loss_provider import LossProvider
import utils_new_mod
import os
import tempfile
# from PIL
# from calculatefid import calculate_fid
from pytorch_fid.fid_score import InceptionV3, calculate_frechet_distance
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import torchvision.utils as vutils
from torch_fidelity import calculate_metrics
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Writing headers.py


In [9]:
%%writefile centerembed.py
import headers
import torch
def embed_watermark_in_center(imgs_z: torch.Tensor, watermark_z: torch.Tensor,alpha:float=0.9):
    """
    Embeds the watermark into the center of the latent space.
    
    Args:
        imgs_z: Latent representation of the cover images (b, z, h/f, w/f)
        watermark_z: Latent representation of the watermark (b, z, h_wm/f, w_wm/f)
    # 
    Returns:
        Modified latent space with the watermark embedded in the center.
    """
    # Get shapes of latent and watermark
    b, z, h_lat, w_lat = imgs_z.shape
    _, _, h_wm, w_wm = watermark_z.shape

    # Calculate center coordinates
    center_h = h_lat // 2
    center_w = w_lat // 2

    # Determine the region in the latent space where the watermark will be embedded
    h_start = center_h - h_wm // 2
    h_end = center_h + h_wm // 2
    w_start = center_w - w_wm // 2
    w_end = center_w + w_wm // 2

    # Embed watermark into the center of latent space
    imgs_z[:, :, h_start:h_end, w_start:w_end] = (
        (1 - alpha) * imgs_z[:, :, h_start:h_end, w_start:w_end] + alpha * watermark_z
    )

    return imgs_z

Writing centerembed.py


In [23]:
%%writefile txt2img.py
import argparse, os
import cv2
import sys
sys.path.append('src')
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange
from torchvision.utils import make_grid
from pytorch_lightning import seed_everything
from torch import autocast
from contextlib import nullcontext
# from imwatermark import WatermarkEncoder
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from ldm.models.diffusion.dpm_solver import DPMSolverSampler

torch.set_grad_enabled(False)

def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

def insert_watermark_in_latents(samples, watermark_latent, alpha=0.05):
    """
    Inserts an imperceptible watermark into the latent representation.

    Args:
        samples (torch.Tensor): The latent representation of the image (B, C, H, W).
        watermark_latent (DiagonalGaussianDistribution): The latent representation of the watermark.
        alpha (float): Scaling factor for the watermark (default: 0.1, adjust as needed).

    Returns:
        torch.Tensor: The modified latent representation with the imperceptible watermark.
    """
    # Ensure watermark_latent is sampled correctly
    if hasattr(watermark_latent, 'sample'):
        watermark_latent_tensor = watermark_latent.sample()  # Call sample as a method
    else:
        raise TypeError(f"Expected a DiagonalGaussianDistribution, got {type(watermark_latent)}")

    # Get the shape of the watermark latent tensor
    wm_h, wm_w = watermark_latent_tensor.shape[2], watermark_latent_tensor.shape[3]

    # Compute the center of the image latent tensor
    center_h, center_w = samples.shape[2] // 2, samples.shape[3] // 2

    # Determine the position to insert the watermark
    start_h, start_w = center_h - wm_h // 2, center_w - wm_w // 2

    # Blend the watermark latents into the image latents
    samples[:, :, start_h:start_h+wm_h, start_w:start_w+wm_w] += alpha * watermark_latent_tensor

    return samples


def load_model_from_config(config, ckpt, device=torch.device("cuda"), verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    if device == torch.device("cuda"):
        model.cuda()
    elif device == torch.device("cpu"):
        model.cpu()
        model.cond_stage_model.device = "cpu"
    else:
        raise ValueError(f"Incorrect device name. Received: {device}")
    model.eval()
    return model

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--prompt",
        type=str,
        nargs="?",
        default="a professional photograph of an astronaut riding a triceratops",
        help="the prompt to render"
    )
    parser.add_argument(
        "--outdir",
        type=str,
        nargs="?",
        help="dir to write results to",
        default="outputs/txt2img-samples"
    )
    parser.add_argument(
        "--steps",
        type=int,
        default=50,
        help="number of ddim sampling steps",
    )
    parser.add_argument(
        "--plms",
        action='store_true',
        help="use plms sampling",
    )
    parser.add_argument(
        "--dpm",
        action='store_true',
        help="use DPM (2) sampler",
    )
    parser.add_argument(
        "--fixed_code",
        action='store_true',
        help="if enabled, uses the same starting code across all samples ",
    )
    parser.add_argument(
        "--ddim_eta",
        type=float,
        default=0.0,
        help="ddim eta (eta=0.0 corresponds to deterministic sampling",
    )
    parser.add_argument(
        "--n_iter",
        type=int,
        default=1,
        help="sample this often",
    )
    parser.add_argument(
        "--H",
        type=int,
        default=512,
        help="image height, in pixel space",
    )
    parser.add_argument(
        "--W",
        type=int,
        default=512,
        help="image width, in pixel space",
    )
    parser.add_argument(
        "--C",
        type=int,
        default=4,
        help="latent channels",
    )
    parser.add_argument(
        "--f",
        type=int,
        default=8,
        help="downsampling factor, most often 8 or 16",
    )
    parser.add_argument(
        "--n_samples",
        type=int,
        default=1,
        help="how many samples to produce for each given prompt. A.k.a batch size",
    )
    parser.add_argument(
        "--n_rows",
        type=int,
        default=0,
        help="rows in the grid (default: n_samples)",
    )
    parser.add_argument(
        "--ldm_decoder_ckpt",
        type=str,
        default=None,
        # default="/kaggle/input/forstatistics/checkpoint_009.pth",
    )
    parser.add_argument(
        "--scale",
        type=float,
        default=3.0,
        help="unconditional guidance scale: eps = eps(x, empty) + scale * (eps(x, cond) - eps(x, empty))",
    )
    parser.add_argument(
        "--from-file",
        type=str,
        help="if specified, load prompts from this file, separated by newlines",
    )
    parser.add_argument(
        "--config",
        type=str,
        default="configs/stable-diffusion/v2-inference.yaml",
        help="path to config which constructs model",
    )
    parser.add_argument(
        "--ckpt",
        type=str,
        help="path to checkpoint of model",
    )
    parser.add_argument(
        "--seed",
        type=int,
        default=42,
        help="the seed (for reproducible sampling)",
    )
    parser.add_argument(
        "--precision",
        type=str,
        help="evaluate at this precision",
        choices=["full", "autocast"],
        default="autocast"
    )
    parser.add_argument(
        "--repeat",
        type=int,
        default=1,
        help="repeat each prompt in file this often",
    )
    parser.add_argument(
        "--device",
        type=str,
        help="Device on which Stable Diffusion will be run",
        choices=["cpu", "cuda"],
        default="cuda"
    )
    parser.add_argument(
        "--torchscript",
        action='store_true',
        help="Use TorchScript",
    )
    parser.add_argument(
        "--ipex",
        action='store_true',
        help="Use Intel® Extension for PyTorch*",
    )
    # parser.add_argument(
    #     "--bf16",
    #     action='store_true',
    #     help="Use bfloat16",
    # )
    opt = parser.parse_args()
    return opt

def main(opt):
    seed_everything(opt.seed)

    config = OmegaConf.load(f"{opt.config}")
    device = torch.device("cuda") if opt.device == "cuda" else torch.device("cpu")
    model = load_model_from_config(config, f"{opt.ckpt}", device)
    # Parameter None for clutil sweep
    print(f'reload decoder weights {opt.ldm_decoder_ckpt}...')
    if opt.ldm_decoder_ckpt is not None and opt.ldm_decoder_ckpt.lower() == "none":
        opt.ldm_decoder_ckpt = None
    if opt.ldm_decoder_ckpt is not None:
        state_dict = torch.load(opt.ldm_decoder_ckpt)['ldm_decoder']
        # state_dict = torch.load(opt.ldm_decoder_ckpt)['state_dict']
        # state_dict = {k.replace('first_stage_model.', ''): v for k, v in state_dict.items() if 'decoder' in k or 'post_quant_conv' in k}
        msg = model.first_stage_model.load_state_dict(state_dict, strict=False)
        model.eval()
        print(msg)

    if opt.plms:
        sampler = PLMSSampler(model, device=device)
    elif opt.dpm:
        sampler = DPMSolverSampler(model, device=device)
    else:
        sampler = DDIMSampler(model, device=device)

    os.makedirs(opt.outdir, exist_ok=True)
    outpath = opt.outdir
    batch_size = opt.n_samples
    n_rows = opt.n_rows if opt.n_rows > 0 else batch_size
    if not opt.from_file:
        prompt = opt.prompt
        assert prompt is not None
        data = [batch_size * [prompt]]
    else:
        print(f"reading prompts from {opt.from_file}")
        with open(opt.from_file, "r") as f:
            data = f.read().splitlines()
            data = [p for p in data for i in range(opt.repeat)]
            data = list(chunk(data, batch_size))

    sample_path = os.path.join(outpath, "sampleswater50k")
    os.makedirs(sample_path, exist_ok=True)
    sample_count = 0
    base_count = len(os.listdir(sample_path))
    grid_count = len(os.listdir(outpath)) - 1

    start_code = None
    if opt.fixed_code:
        start_code = torch.randn([opt.n_samples, opt.C, opt.H // opt.f, opt.W // opt.f], device=device)
        
    if opt.torchscript or opt.ipex:
        transformer = model.cond_stage_model.model
        unet = model.model.diffusion_model
        decoder = model.first_stage_model.decoder
        additional_context = torch.cpu.amp.autocast() if opt.bf16 else nullcontext()
        shape = [opt.C, opt.H // opt.f, opt.W // opt.f]

        if opt.bf16 and not opt.torchscript and not opt.ipex:
            raise ValueError('Bfloat16 is supported only for torchscript+ipex')
        if opt.bf16 and unet.dtype != torch.bfloat16:
            raise ValueError("Use configs/stable-diffusion/intel/ configs with bf16 enabled if " +
                             "you'd like to use bfloat16 with CPU.")
        if unet.dtype == torch.float16 and device == torch.device("cpu"):
            raise ValueError("Use configs/stable-diffusion/intel/ configs for your model if you'd like to run it on CPU.")

        if opt.ipex:
            import intel_extension_for_pytorch as ipex
            bf16_dtype = torch.bfloat16 if opt.bf16 else None
            transformer = transformer.to(memory_format=torch.channels_last)
            transformer = ipex.optimize(transformer, level="O1", inplace=True)

            unet = unet.to(memory_format=torch.channels_last)
            unet = ipex.optimize(unet, level="O1", auto_kernel_selection=True, inplace=True, dtype=bf16_dtype)

            decoder = decoder.to(memory_format=torch.channels_last)
            decoder = ipex.optimize(decoder, level="O1", auto_kernel_selection=True, inplace=True, dtype=bf16_dtype)

        if opt.torchscript:
            with torch.no_grad(), additional_context:
                # get UNET scripted
                if unet.use_checkpoint:
                    raise ValueError("Gradient checkpoint won't work with tracing. " +
                    "Use configs/stable-diffusion/intel/ configs for your model or disable checkpoint in your config.")

                img_in = torch.ones(2, 4, 96, 96, dtype=torch.float32)
                t_in = torch.ones(2, dtype=torch.int64)
                context = torch.ones(2, 77, 1024, dtype=torch.float32)
                scripted_unet = torch.jit.trace(unet, (img_in, t_in, context))
                scripted_unet = torch.jit.optimize_for_inference(scripted_unet)
                print(type(scripted_unet))
                model.model.scripted_diffusion_model = scripted_unet

                # get Decoder for first stage model scripted
                samples_ddim = torch.ones(1, 4, 96, 96, dtype=torch.float32)
                scripted_decoder = torch.jit.trace(decoder, (samples_ddim))
                scripted_decoder = torch.jit.optimize_for_inference(scripted_decoder)
                print(type(scripted_decoder))
                model.first_stage_model.decoder = scripted_decoder

        prompts = data[0]
        print("Running a forward pass to initialize optimizations")
        uc = None
        if opt.scale != 1.0:
            uc = model.get_learned_conditioning(batch_size * [""])
        if isinstance(prompts, tuple):
            prompts = list(prompts)

        with torch.no_grad(), additional_context:
            for _ in range(3):
                c = model.get_learned_conditioning(prompts)
            samples_ddim, _ = sampler.sample(S=5,
                                             conditioning=c,
                                             batch_size=batch_size,
                                             shape=shape,
                                             verbose=False,
                                             unconditional_guidance_scale=opt.scale,
                                             unconditional_conditioning=uc,
                                             eta=opt.ddim_eta,
                                             x_T=start_code)
            print("Running a forward pass for decoder")
            for _ in range(3):
                x_samples_ddim = model.decode_first_stage(samples_ddim)

    precision_scope = autocast if opt.precision=="autocast" or opt.bf16 else nullcontext
    with torch.no_grad(), \
        precision_scope(opt.device), \
        model.ema_scope():
            all_samples = list()
            for n in trange(opt.n_iter, desc="Sampling"):
                for prompts in tqdm(data, desc="data"):
                    uc = None
                    if opt.scale != 1.0:
                        uc = model.get_learned_conditioning(batch_size * [""])
                    if isinstance(prompts, tuple):
                        prompts = list(prompts)
                    c = model.get_learned_conditioning(prompts)
                    shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                    samples, _ = sampler.sample(S=opt.steps,
                                                     conditioning=c,
                                                     batch_size=opt.n_samples,
                                                     shape=shape,
                                                     verbose=False,
                                                     unconditional_guidance_scale=opt.scale,
                                                     unconditional_conditioning=uc,
                                                     eta=opt.ddim_eta,
                                                     x_T=start_code)
                    # watermark_image = Image.open('/kaggle/input/mnistasjpg/testSet/testSet/img_100.jpg').convert('RGB')\  # assuming RGB for color watermark
                    watermark_image = Image.open('/kaggle/input/fashion-mnist-png/train/1/10012.png').convert('RGB')
                    watermark_image = watermark_image.resize((32, 32), Image.Resampling.LANCZOS)
                    watermark_tensor = torch.tensor(np.array(watermark_image), dtype=torch.float32)  # Normalize to [0, 1]
                    watermark_tensor = watermark_tensor.permute(2, 0, 1).unsqueeze(0)  # Add batch dim and change channel order to CxHxW
                    # watermark_tensor = watermark_tensor.unsqueeze(0).unsqueeze(0)
            # Pass the watermark through the encoder to get the latent representation
                    watermark_latent = model.encode_first_stage(watermark_tensor.to(opt.device))  # Assuming an encoder method exists
            # Insert the watermark latent into the samples
                    modified_latents = insert_watermark_in_latents(samples, watermark_latent)
                    # x_samples = model.decode_first_stage(samples)
                    if opt.ldm_decoder_ckpt is not None and opt.ldm_decoder_ckpt.lower() == "none":
                        x_samples = model.decode_first_stage(samples)
                    else:
                        x_samples = model.decode_first_stage(modified_latents)
                    x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                    for x_sample in x_samples:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                        img = Image.fromarray(x_sample.astype(np.uint8))
                        # img = put_watermark(img, wm_encoder)
                        img.save(os.path.join(sample_path, f"{base_count:05}.png"))
                        base_count += 1
                        sample_count += 1

                    all_samples.append(x_samples)

            # # additionally, save as grid
            # grid = torch.stack(all_samples, 0)
            # grid = rearrange(grid, 'n b c h w -> (n b) c h w')
            # grid = make_grid(grid, nrow=n_rows)

            # # to image
            # grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
            # grid = Image.fromarray(grid.astype(np.uint8))
            # # grid = put_watermark(grid, wm_encoder)
            # grid.save(os.path.join(outpath, f'grid-{grid_count:04}.png'))
            # grid_count += 1

    print(f"Your samples are ready and waiting for you here: \n{outpath} \n"
          f" \nEnjoy.")

if __name__ == "__main__":
    opt = parse_args()
    main(opt)

Overwriting txt2img.py


In [12]:
%%writefile for1k.txt
A teddy bear sitting next to a picture of himself on a sandy beach and other travel memorabilia.
A large variety of apples lined up at the store.
Gray cat with green eyes looking at camera while lying on a pink toy.
dried flowers extend from the opening of a black vase with a white brick wall in the background.
Two skiers on a field with the sun shining.
a person sitting on a ledge with a bike near by
An open clean, white bidet is on display.
something round being cooked in a shape of a pancake in a fry pan
Blue seats in a bus in a city.
A table with a couple of bears on it.
Three sheep walk down a path behind a man. 
A tall sandwich with pickle on a white plate.
a public transit bus on a city street 
Chicken with chies and red peppers, rice, carrots and broccoli.
Looking down at a desktop with two monitors, a key board, mouse and cell phones on it.
The people in the photo are all reaching your hands up high.
A image of a cat at the doorstep looking outside
A man on a surfboard riding a wave.
A man holding a banana in his hand.
A small child is holding a teddy bear.
There is a crossing sign next to the car. 
this bathroom has a wooden cabinet and a sink on top
A knife is sitting on a plate of food
A child standing next to a bed with a laptop.
a man in a black suit gazing a round
a vase with a flower growing very well
A guy in a black shirt charging his cell phone.
A group of people that are holding onto sheep.
A black cat wears a bumblebee hat on its head. 
Bridge with arches over lake with metropolitan buildings in background
There is a bird sitting on a tree branch.
A bird on the edge of a boat in a body of water.
There is a plate of food and a pizza on a table.
An old time car is parked at the curb near a stop sign.
The tennis player's ball is in mid air.
a wood bench is outside covered in snow
A large pizza sitting on top of a pizza pan.
A bench in th side of a city street next to a window
A black cay laying on top of a suitcase. 
a fat girl blowing out candles on a cake
A man has thrown a frisbee into a metal structure.
a brown donut on some white paper on a red counter
a person in a red top is laying on a brown couch
The zebra stands and looks down at the ground.
Group of spoiled fruit sitting next to each other. 
Two people on hose back riding through a fair while a woman in the foreground talks on a cellphone.
A jet that is flying in the sky.
A small group of construction workers by a crosswalk.
A person bicycles along the edge of the road overlooking the river.
Two giraffes are in the grass by some rocks.
A street with sign, lamppost and many trees.
Two people dressed in black, one with a green mask.
a man on a cell phone holding a microphone up to his ear
a man in a chefs outfit standing by a big donut
The horse is eating grass from the ground.
A black cat staring at the camera taking the picture.
A blurred train is going under a bridge.
Two screens show two unique pairs of wings.
a woman with eye glasses in front of a pair of birds
Toilet in a small room with plant pots.
a close up of a parking meter 
A kitchen slab on which there is a microwave and few cups are kept in a tray. 
A baby elephant walks between the legs of its mother.
A clean knife and french bread sit on a cutting board.
A baseball batter is swinging a bat at an incoming pitch.
An open partially packed suitcase resting on the floor with a cell phone next to it.
A car is stopped at a red stop light.
a hand is holding a knife that is cutting some cake
some giraffes a building trees rocks dirt and plants
a tiled floor with a toilet sitting on top of it 
Automobiles stopped at an intersection because of a passing train. 
A girl in plain dress standing next to a white teddy bear.
A man bends over in the snow in sight of a mountain range.
A cutting board with broccoli being prepared for a meal.
A foot that is on top of a banana.
Two stuffed bears have shirts on their heads. 
some people on skis ride on the snow 
Old baseball bats are turned into a seat.
a room showing a very big fridge and a dining table
A rusted out train engine sitting next to a green forest.
A woman walking with a man and holding an umbrella looks off to the side. 
A young girl holding a piece of food and her hand over her mouth.
A skateboarder preforming tricks on a half pipe. 
A man is leaning back talking on the phone
A group of bikers riding bikes down a street.
a giraffe is standing in a grassy field 
Four zebras and a nearby giraffe on some grass.
A man sitting inside an outdoor fruit market.
a close up of a zebra behind a fence 
A suit case is opened and there are things cluttered around
a red and yellow fire truck and some buildings
stream running beside a railroad track in the mountains.
A tennis player prepares to hit a ball during a match.
A little brown bird sitting by a window.
The light shines on a cluttered desk with a laptop on it.
The large cargo train is green and yellow.
Four people are flying kites on a beach.
A plane that is taking off at an airport
A group of ninjas wearing all black hold up small white fans.
A bathroom with urinals that have graffiti above them
A cat laying on top of a computer keyboard.
some kind of nasty bathroom that I would never use.
A baby lays inside of a small, open suitcase.
skiers riding on a ski lift to the top of a mountain
Two women are smiling as one of them holds a menu.
A man in uniform with a cell phone up to his ear.
Two very cute fuzzy young sheep on  a grassy hill.
a lady with green hair sitting next to a horse
A person sitting down with their legs stretched in front of them, on a skateboard ramp with a skateboard several feet in front of them.
there is a small vase with many tulips in it
A man in a wet suit surfs a wave.
A group of women running after a soccer ball
Two adults with umbrellas and two children walking in the rain. 
Two sun shaped mirrors are above bowl shaped sinks.
A group of men in red vests rowing a dragon boat.
A group of people in the woods with dogs.
Buildings with a clock and a crane over head are in this picture. 
People mingling on a beach with a kite floating in the sky.
A cat laying on a bed next to a laptop.
A sleeping grey husky wearing a green bandana.
A batter getting ready to hit his pitch.
A plate containing a slice of dessert, two forks and some piped cream
a lady eating at a table holding up the peace sign
A crooked one way sign pointing into the ground
A couple of slices of a cinnamon roll on a white plate.
A park bench made out of wood in the park. 
A picture of a young woman holding hands with a dog.
A dog with its paw on a remote controller on a couch.
A baseball player running the bases at a game 
the bathroom has a mirror and a tiled sink
Several elephants in the water with one up close standing next to a trainer 
A rear view mirror with a large red semi truck.
A little child standing next to a toy dump truck.
A toothbrush being held under a water facet.
Black parking meter on sidewalk in urban street setting.
An orange cat rests on a white counter top
Peeled and cut  plantains are getting coated with seasons .
A bus turning at the intersection of a city street
A white bathroom mirror mounted to it's wall.
A group of people walking down a  street on a wet sidewalk.
A cat that is sitting on some cushions.
People walk through the street with umbrellas to shade the sun.
Some very cute sheep standing in a room with smiling faces. 
Many people play sports in a grass field. 
a public transit bus on a city street
A flock of birds flying in an overcast sky
a small bird on the ground in a field
The giraffe is looking into the back window of a car. 
A half eaten hot dog being held in a hand at a baseball game.
An outdoor kitchen area with chefs and a stove.
A umbrella that has multiple different colors on it.
A remote control has all black and white buttons except for a yellow "Light" button.
Red bus with graffiti stops to pick up passenger.
a big bathroom with a very large bath tub
A man on a skateboard is jumping over a pile of blocks.
This is a very ornate building with a clock on it.
A young woman poses comically with a piece of pizza in her mouth.
A man in a large room with baskets and pottery
Sepia photograph of small elephants in a wildlife park
Two people are sitting on a bench together in front of water.
A woman with a bowl of food and seated
A refrigerator full of juice and soda next to a microwave. 
a gray and white bathroom with a sink mirror and toilet
Two white birds are on an elephant's back.
A large stork spreads it's wings as it lands.
A girl flying a kite on the beach.
A brown teddy bear in a forest with trees and shrubbery.
A man wearing a red hat and a suit and tie.
a red bus is in front of another one
Some people are standing on a crowd crowded sidewalk
A clock is sitting at the top of the tower.
The signs give the street names and show where to park.
A surfer wearing red shirt surfing on a wave.
A green train traveling through a train yard.
A clock on top of a signpost stands in front of a building.
A young man sitting in front of a white piano.
a street sign on the corner of a city street 
A man and a woman posing together for a picture
The large train is black, yellow, and red.
A person in a mid air jump while skiing on a snowy slope.
A kitchen area with refrigerator, counter and a microwave.
a bunch of cement blocks stacked side by side in front of umbrellas
The young skateboarder is trying one of his first tricks.
The bird is walking beside a bull with dark horns.
A toilet is in a dirty bathroom with a sink.
A person riding a skateboard next to the ocean.
a small toilet in a bathroom in a portable vehicle
A dog has crazy eyes as he tries to keep the frisbee.
A park at night is shown, with an empty bench centered
A man standing in front of a table with a plate of oysters.
one laptop two monitors two keyboards and two mice
A small room built from legos with a tiled floor.
a big sign sits above a building 
A white toilet sitting inside of a bathroom stall.
A ram stands alone in grass between large rocks.
A table has three wine glasses, several bottles of wine, and a bowl of salad.
A little girl with a baseball bat is waiting for the pitch.
There is a Christmas tree in a living room.
A train is traveling down the railroad tracks. 
A very primitive ferry holds a truck and three occupants.
A woman is outside preparing to fly a kite
a number of people in a room with a person playing nintendo wii
a collage of photos with four fire hydrants 
A collection of various street signs by a big building.
A couple of birds fly through a blue cloudy sky.
a woman biting into a large slice of cheese pizza 
A guy on a skate board on a big ramp.
Two women, one with a black helmet and one with a red helmet sit on a white scooter.
A man is cross country skiing in a field.
A group of cattle are gathered under a shade tree.
An elephant is looking out through a thick rope fence.
An elephant that is very close to the camera. 
A cobblestone city street lined with buildings on a sunny day
A man riding skis down a snow covered mountain.
a silver and black stove and microwave a sink and cabinets
a herd of giraffes eat on some tree leafs 
a couple of sheep walking across a grass covered field.
People on corner with cars on street and stoplight with building
A stoplight with a road sign on it 
This is an outdoor restroom that is handicap accessible. 
Lamps are the only lighting in this kitchen. 
a cat is laying on a window ledge
A parking garage, with a green painted floor, full of scooters
Two Nintendo Wii remote controllers that are white with one of them on a stand.
a woman holding a plate with some colorful slices of cake on it
Man in white shirt holding up a cellphone in a crowd. 
TABLE WITH A PLATE OF FOOD A DRINK, CELL PHONE AND A COMPUTER
There is a plate filled with pasta and vegetables.
The dual image shows people carrying their surfboards under their arms.
A black dog in a yard jumps up toward a yellow Frisbee.
Stores, people, street signs and cars on a street.
A Metro bus line departs its terminus station
A fridge with its doors opens stands in a room.
a number of cars on a street with traffic lights 
Large assortment of vegetables displayed for sale on large round table.
Three surfers in wetsuits going in to the ocean
a couple of people that are standing up by a surfboard
Rows of boats are docked along a waterway.
An umbrella is placed inside an old microwave. 
A group of people dressed in an elephant costume.
A bathroom with two toilets and a shower in it, with no space for any of them.
The white cat is sitting underneath an umbrella 
A man in glasses is wearing a tie with extraterrestrials on it.
A large marble bathroom with mirrored closet and jetted tub.
 A man wearing a dress shirt and a tie sitting against a dark background. 
A pole on a city street covered with signs and stickers in front of a blue crane.
A person that is holding an umbrella in her left hand.
A person kiteboarding over waves in the ocean.
A still image multi shot of a man on a skateboard doing tricks on the ramp
A chocolate donut and cup of coffee on a napkin.
The locomotive of a train is painted yellow.
A street with a traffic light under a cloudy sky.
a close up of two people playing soccer
Several people on skis either standing, walking or skiing down the slope. 
The sweet dish features bananas on top of the crust..
A bus is parked and waiting on passengers 
A man holds a black plate with a square skull in front of his face.
Bicycle rick shaw with umbrella and rain canopy
A man flying a kite over a snow covered field.
A small black and brown dog standing next to a cow.
An object is seen here in this unique image.
A made bed in a hotel, with a mirror on the wall behind it.
A person holding a cell phone in their hands.
A young bride smiles at the crowd while she and her groom cut the cake. 
a number of horses in a field with a sky background
A pizza cutter is designed to look like the Enterprise
A zebra standing beside a tree at the zoo.
A man sitting on the park bench in the forest 
A huge clock on the side of a building.
A white toilet is separated from a brown shower curtain.
 a close up of a clock face saying it is a little after 2 o'clock
A baseball player followed by a small child walking on a baseball field 
A pole with signs on it in the street out side.
A man jumping in the air in a horizontal direction over the top of a bed.
A laptop computer is on a glass table on a balcony.
A man is approaching a seagull on the beach.
Bowls of food are sitting on a table.
A basic cafeteria breakfast on a plastic tray
A clock sitting next to a brick sign under palm trees.
A batter swings at a pitch during a baseball game.
A couple of chairs that are in a room.
A man is enjoying a day snow skiing. 
someones room with a purple bed and paper everywhere 
A smiling woman looks to be playing a Wii.
A man that is laying down in a bed.
A large bus carrying several passengers drives through a busy part of town
A little girl is getting ready to blow out a candle on a small dessert.
Four boxes of pizza are opened on the table
The baseball players slides into safety at home plate.
A blue crew cab truck sitting in the parking lot of an apartment building.
A brown cow sits in the middle of a grassy field.
The young man is sitting on the suitcases relaxing. 
two people sit on a bench behind signs for Carnell Pitts avenue and Cory Hatter avenue
A street sign displaying multiple cities and their directions.
A bathroom with a walk in shower currently under repair.
Two zebras in an open field with grass.
A cat stretching its paw over a keyboard. 
a plaza filled with a lot of birds and some cows
a black and white dog is in a cage
Airplane low in the area near an air traffic control tower.
Bunch of apples and oranges sitting on a white surface.
A horse is standing in the grasses of a field. 
Two trains with grass and blue skies and power wires.
The entire baseball team has gathered on the field for a celebration. 
The young boy is playing in the living room of his house. 
a couple of people that are dressed up dancing
a man standing while holding onto a tennis racket in his right hand
Two zebras running on a path next to trees.
A small square plate of broccoli with seeds and a scoop of rice
A large airplane flying over a lush green hillside.
A man on a skateboard flipping the skateboard in the air.
A man who is diving to hit a tennis ball.
A cat with it's paw next to a computer mouse on a wooden table.
two laptop computers sit on a countertop, plugged with wires
Three monitors play a movie on the computer desk.
Two young kids playing with letters in the kitchen.  
A dog sitting in a basket mounted on the front of a blue bike.
A horse and several cows feed on hay.
A baseball game is being played before a crowd.
This shot is of a crowded highway full of traffic
A large jetliner flying over a small farm near a forest.
A man holds a balloon and stuffed bear over the edge of a balcony.
Virgin sponsored airplane parked with a plane leaving in the background.
A polar bear is in the water holding on to a disk.
A train carrying freight containers is on the tracks.
There is a scrunchie on top of a laptop computer.
A little boy sits in front of several tortilla pizzas.
a kitchen with a nasty dirty kitchen with a light beam through the window
The caramel-colored kitten is  lying on the small blanket on top of the unmade bed.
A man that is sitting down with a cellphone.
Several children attentively play a game of curling together.
A modern and clean bathroom with a big sink.
A puppy has pulled toilet paper across the bathroom floor.
A cat sitting on top of a towel.
Two men are competing in cross country skiing. 
a woman standing with an umbrella over her head 
A crowd of people are gathered in a grassy park.
Two bears standing on a grassy hill facing each other. 
Shot of various traffic signs and traffic light in city area.
A man skis over a plastic contraption on a ski slope. 
Three elephants standing in the grass near water.
A man in a yellow shirt is surfing through the waves.
A couple posing on a horse statue in a courtyard.
Looking down at a partially eaten salad sandwich in paper
Person holding open a large door to the walk-in refrigerator
The man on the phone is looking at the camera and smiling.
A train traveling on the tracks approaching a station.
An Asiana Airlines plane taxiing at an airport.
A bear shaped cake with lit candles inserted.
An assortment of broccoli and cauliflower is arranged.
A moving truck painted with graffiti is in front of a bank.
A man holding out two hands as if pulling on something
A man uses the computer while sitting at a table.
A train in a city with tall buildings.
A bed sitting in a bedroom next to a window under pictures.
this giraffe looks like it is kissing the tree limb
A group of men who are posing for a picture together.
a computer and speakers on a wood desk
A woman in plaid pants posing for a picture on skis.
Two women are working on a dessert together
A small bathroom with a white vanity and a glass shower.
there is a big dog that is sitting inside of a luggage
this is a man throwing a disc on the beach
A coffee cup is parked in between two digital parking meters.
A bird standing alone in the water looking
A person is looking in the mirror of a motor bike
the bathroom has a toilet and a sink and mirror
A man riding a bike down a dark city street.
A variety of people stand in a field by a monument.
A messy room with a laptop on a bed.
A green double-decker bus parked at a bus stop. 
A big full view of several people gathering.
People are skiing down a long ski slope.
A transit bus driving down a highway by some fast food restaurants.
A man that is laying down on a bed.
A man eats something that is wrapped in white paper.
A big fancy clock handing from the ceiling.
A female sitting on her bed looking at a laptop.
A man paints scenes at a fair stand.
Three giraffes huddled together watching a zebra and gazelle.
some people are riding surf boards in rapids
A bathroom with a sink, toilet and tub in front of a large mirror.
THERE ARE CHRISTMAS DECROATION ALL OVER THE PLACE
A picture of some people playing with a frisbee.
a large statue of two people standing near a luggage bag
A boy at a skate park practicing on his skateboard. 
A group of people pose with three bicyclists. 
Horses grazing in a muddy portion of a flooded field
A table with a piece of construction paper, scissors and sewing thread.
a man standing next to sheep on a lush green park.
A couple of elephants standing on top of a grass covered field.
The young person is surfing on wave in the sea.
Two parking meters on a city street. 
A small bathroom, lacking only a toilet tissue dispenser.  
A couple of yak in their field eating some grass.
The basketball players are playing in a game
Someone is eating a piece of sandwich next to a glass.
Some statues of elephants are on blue platforms.
Firefighters are riding in the back of a fire truck.
A harbor filled with boats surrounded by buildings.
a close up of a child wearing a shirt and tie kneeling down
Two horses grazing in a field near a lighthouse.
Television monitor in room with many items displayed.
A man on a raft waving a jacket and a surfer in a wet suit out in the ocean
Giraffes in a zoo, standing with their heads peeking over the fence, looking at people.
A man touching home plate on a baseball field.
People on surfboards in the blue sea, a woman lying down and a man paddling with an oar.
Two large adult cats relaxing on a bed.
A man talking on a cell phone wearing an orange shirt and baseball cap.
A white vase with white flowers and two candles on a table.
Looking through a door and seeing a toilet and sink.
A view form under a tree of a window
a red and white fire hydrant is sitting by a curb
A cityscape of a set of train tracks with trains running on them.
A young soldier has his picture taken with a young lady with an open jacket and brazier showing.
Small white bathroom with marble counter tops on the sink. 
black and white image of a bench in a forest
A very small kitchen in an attic with a sink  
There is a somewhat short train moving along a track
A polar bear buries its head under the rump of another polar bear.
A guy in funny hat holding a very big bird.
a plate of broccoli on a table next to some other plates of food 
Groups of people moving down road on skis
A man is asleep in bed with a laptop open on his lap.
a childrens soccer game in the background, one kid is kicking the ball
A city fire hydrant next to a taxi cab.
A plane that is flying in the sky over two houses.
Pedestrians hold umbrellas as they cross the street as snow starts to fall.
a girl that is on a tennis court with a racket
Girl using her laptop computer while laying on a bed.
Two girls are posing outdoors under an umbrella for a photo. 
People approaching a line of buses at a busy terminal.
A jet shoots up into the cloudy sky.
A frame has six pictures, two with a horse.
A sign post with signs that read "Maciel Ln" and "Wonder Stump Rd".
Man photographing a fresh pizza his tired buddy just wants to eat
A person holding a smart phone in their left hand.
A person is riding waves on a canal.
A blue street sign marks Hollywood Boulevard. 
A group of zebra's eating hay from a trough.
A man standing in a tunnel next to a car.
A man holding a red snowboard while standing in snow.
A group of giraffes eating leaves off trees.
A dog wrangling a group of sheep in a field.
A man doing a trick on a wall with a skateboard.
A picture of a baseball player from a long time ago.
a fence with a mountain in the background
A man driving a luggage cart sitting on top of a runway.
Bottles and other items on a counter top.
There are two people who are out in the snow.
A group session of children reading books to their dogs.
A man walking with a drink and a bag while listening to earbud headphones.
A little boy in a green shirt holding a purple toy.
A man in a coat in tie wearing a skirt and stockings. 
A man surfs a small wave in calm waters.
A surfer is riding a wave in light blue water.
Two men smile while sitting at a table outdoors.
A man standing on a type of boat during the day.
A hot dog in tin foil with mustard, onions and other toppings.
A hillside near the ocean with a large wall and possibly a road constructed into the hill.
some horses standing by a trailer in the snow 
A clock that is sitting behind a window.
A zebra stands next to rocks by himself. 
A man riding a surfboard while flying a kite.
A couple of men walking along a river.
A black and white photo shows a frisbee disc.
the lines are black and white and run parallel 
A man wearing a purple neck tie and a V neck sweater.
There are three toothbrushes and two bottle of soap on this sink.
a white bird standing on a wooden structure next to a body of water
A young man standing and posing in a Chicago Cubs baseball uniform.
A couple of horses standing on top of a grass covered field.
Two little girls who are sitting on a couch.
A man sits in a dimly lit room using a laptop computer.
A man standing on a mountain top wearing skis.
A cat that is standing looking through a glass.
A female skateboarder is in mid air doing a trick.
people sitting at the beach, playing in the water, and flying a kite 
Man serving tennis ball and racket with camera watching on court
A computer sits on a desk by a printer.
A scared boy holding a baseball glove out towards a baseball.
a man on a giant bicycle rides by a tall pole in front of an empty, large field in front of some mountains
The furniture in the small living room is mostly red.
A young boy is unwrapping paper covering a box.
Many people skateboard through an intersection, stopping traffic.
A kitchen with wood floors and one brick wall.
Cabbage, broccoli and other assorted vegetables are arranged on a table.
Two urinals with foreign cartoons taped on the tile.
A collection of silverware resting on a rack.
Several green and red apples lying on ground.
A laptop computer and a desktop computer on a white desk
Several men playing soccer in a park with a few onlookers.
An electronic device displaying the time sitting on a table.
A woman eating a doughnut sitting at a laptop.
An omelette and side of greens on a plate with coffee and a glass of juice. 
A train with it's doors open sitting next to a platform.
A PINK TOILET AND A PINK BATH TUB IN THE BATHROOM
Two surfers make their way to the water carrying surfboards.
The little boy is on his skateboard riding outside.
A nun riding a skateboard down a sidewalk next to parked cars.
People at a restaurant waiting to be served.
Black and White cat lays inside of the sink
The boy is riding a skateboard off a skate ramp.
A young zebra is sniffing the ground in a dusty area.
A woman holding a plate of cake covered in frosting.
A small cat examines the light in a suitcase.
A couple of sheep standing in the tall grass.
A boy carrying two skateboards crosses the street.
A man has a crow sitting on his hand.
A family of ducks swimming in the water
A group of people and a horse are at the edge of the garden in front of the bluff on a sunny day.
THERE ARE PEOPLE THAT ARE PLAYING FRISBEE ON THE FIELD 
A white plate with lots topped with garlic bread.
an older woman walking in the street and a bus 
a couple of birds swimming in a lake 
A group of animals standing in the grass.
A man in the kitchen holding a beverage. 
a banana tree in the woods with bananas on it
A man wearing a bushel of bananas on his head.
A large clock with flags on it says Colgate.
A kitchen with a refrigerator freezer next to a sink.
A clean bathroom with a tub and sink. 
A hard suitcase with reinforced edges on the floor
A woman on a crowded street with and umbrella and shopping trolly.
A young boy grinds a rail with his skateboard.
Two women bake together at a kitchen island.
A set of four bikers riding motorcycles side by side.
A bottle of water near a computer screen and a keyboard with fingers on it.
A bird on the beach with wings out stretched.
A tray on a table with plates of food and dipping sauces and a drink.
there are many giraffes that are standing by a fence
a black plane is sitting on a runway
A person that is making something out of wood.
A woman wearing a purple jacket is shown on skis.
A woman is standing in the kitchen with another woman.
Zebras and giraffes together in a zoo cage.
A traffic light next to a blue car in front of a building.
an image of a child putting food in its mouth
a desk with bananas, a instrument case and a globe sitting on it.
Traffic signals and sign on pole at roadway intersection.
A female tennis player watching the ball. 
A close up shot of horse, with it's baby in the back.
A family is on a beach and a cruise ship is in the background.
A woman smiles as she stands in skis on a snowy hill.
A man with a jug and a bucket in a kitchen
A road sign sitting in front so a billboard.
Zebras munch on feed in a brick and stucco structure.
A group of jets sitting on top of an airport tarmac.
A collection of antique pocket watches in various condition
A team of girl tennis players gather together at the net for a group picture.
a person wearing a black coat and a tie with bolt designs on it.
A set of women's personal care items sitting on a bed.
Six plates of the same gourmet dish on a wooden table.
Blue and yellow passenger train with its doors open on a station. 
A man riding a skateboard down a ramp.
A horse pulling a cart down a road.
a four engine propeller jet flies through the sky
A picture of two men in the service on a table below a clock. 
A thick sandwich on a plate with a fork and a pickle.
A person walking across a desert filled surrounded by mountains.
A man is swinging a tennis racket at a ball.
A small blue race car passing by a motorcycle
A sugar doughnut on a plate with a tall glass of orange juice.
The giraffe is standing alone in the field.
A woman taking a picture of a woman by several pizza's.
An oblong tray has assorted vegetables on it.
A decorative container with mini roses is sitting on a pedestal. 
A large  white, blue, and red clock shaped like a cup.
Many people are posing for the camera outside. 
A girl standing in a field, wearing a red sweater and a pair of blue jeans
Red train engine holding other cars to it. 
A couple of zebras are standing in a field
A man smiles maniacly while putting a dog into an oven.
A plumbing sign hangs on a pole next to a road.
A woman playing frisbee in a park 
Two children who are on wake boards in the ocean.
two people are walking past a fire hydrant spraying water
A person dressed up as a chef cooking.
Two piles of trunks sitting next to each other.
A umbrella sitting over lawn chairs on a beach.
A skateboarder flips his skateboard as he flies through the air.
a big line of motorcycle that is parked on a concrete 
A woman stands next to a kite in a sunny park.
A large animal laying on top of a dirt and grass ground.
A pair of glasses, a pocket watch and two books.
some people playing in a park with a kite
A person with a football running with another person behind them.
A rumbled bed sits empty as sun shines in the adjacent window. 
A zebra is looking at the camera in a field.
A refrigerator sitting on the inside of a large stone mountain.
a messy bathroom in a fast food restaurant 
A woman is sitting on a bench in front of the water.
A man hitting a tennis ball with a racquet.
A man walking while holding an umbrella on a wet sidewalk.
Girl dressed in a scouting uniform standing near a fence with her foot on a small cannon.
A very large tower like structure with two clocks on it.
The woman playing on the clay court has hit a tennis ball. 
Two rams stand in the snow up to their ankles
a little kid is brushing his teeth and smiling
Camp with make shift outdoor kitchen room 
A living room with large windows facing the forest.
A white toilet in a bathroom next to a trash can.;
A woman with a parasol stands in front of stage.
A small bird sitting on a chair by a table.
A bus is parked on the side of an empty street.
Two open laptops on a desk pointing in different directions. 
Kids are skateboarding at a skate park and one them has fallen down.
A small, black train is on its tracks by a fence.
A mess sits on the floor of an otherwise clean kitchen.
A man in a tiny boat next to many boats with the Canadian flag.
Soccer players during match play in city environment.
A man and dog on a sit upon kayak getting off at a dock
A red truck sitting in a parking lot with the hood up.
A woman riding skis across a snow covered slope.
The pizza is prepared and ready to be eaten
A spotted dog is chasing another dog outside
A person is standing in front of a dog.
A dog with a stuffed animal sitting on a bed.
A group of giraffe Standing up against a dirt wall in front of a crowd of children.
A mother elephant bumping her trunk against her baby's forehead.
A dog sitting in the front seat of a truck.
Two giraffes standing in a field together in front of rocks and trees. 
A vintage teddy bear cuddles in the midst of pillows.
Two men in room playing a game with Nintendo Wii controllers.
Some people out in the water with a sail in the sky.
A quiet highway with a street sign up ahead.
The woman is riding her bike pass the sign with the zebra.
A cute giraffe sitting on the ground in a zoo exhibit. 
there is a man and a woman sitting at a table smiling
a lady that has a umbrella in her hand
A woman wearing an Army t-shirt hits a tennis ball
various flowers and vines sitting on top of a ceramic, circular object
People sitting in a room and watching a small television.
A baseball team playing a baseball game in front of a crowd.
A man is lighting something at the bar table next to his phone. 
A man flying an airplane kite in the sky.
A child is in front of a birthday cake with candles.
A pizza with pepperonis, mushrooms, tomatoes and other various toppings.
A close shot of a toilet with the seat up and some toilet paper. 
an image of a couple playing video game
A red, white and green Amtrak train stopped at a station. 
Two people sit together in a restaurant at a wooden table.
a table that has a vase with a flower in it
A train traveling between two very large rocky mountains.
A woman leaning on a building talking on her cell phone 
The hot dog is on a plate with a pickle next to it.
A desk that has three computers and a printer on it.
A person that is laying on a surfboard in the water.
Assorted colored plastic flowers in a white vase.
A herd of zebras in the wild near grassy terrain.
The father and daughter are under an umbrella on the beach.
A black wii controller  showing from bottom up.
Three large vases with flowers in front of a dining area.
A surfer surfing on the surfboard with an oar in the ocean. 
A brown bear sitting on a bunch of fallen logs
a yellow train parked on the track next to a platform
A woman standing in front of window next to a bug and a stop sign.
pot like models on a table well molded
A woman seated at a table is holding a glass of wine.
A banana tree with bananas hanging from its branches
a picture frame a cup with flowers resting on the desk
A guy does a skate board trick while people watch. 
A batter for the Boston Red Sox walking on the field.
A group of people work in a kitchen together.
a snowboarded catching air jumping off a slope 
Close up of a street pole with a stop sign with graffiti under a one way sign under two street signs with low buildings, telephone poles and wires behind and a blue sky with clouds.
A person stands in the snow holding a snowboard.
A red stop sign sitting on the side of a road.
Cat catty sitting on a suitcase with a closed kennel against the wall.
A brightly lit restaurant on the side of the road at night.
Someone in a room taking a picture with phone out a barred window.
A woman is holding an open red umbrella over her head.
A truck traveling down a highway road filled with oranges.
A pile of books scattered across a naked mattress.
A plate of food and some cups of drink on a table.
A public clock with ornate art deco figurines of women supporting it
A man that is on skis in the snow.
A young man looks delighted with his shiny snowboard.
a cat sleeping on the floor with a little toy
A group of people standing on top of surfboards next to the ocean.
A road sign on top of a stop sign 
A plate of pasta, mushrooms, broccoli. and cheese.
A city block with people, cars, buses and a street vendor. 
A tablecloth has children's writing on it and a pizza.
Men playing with a frisbee in a dry field 
An baby elephant is nudged by an older elephant in front of a stilt house. 
An umbrella with a cat sitting underneath it.
Three surfers and one swimmer watching a beginner surf
a black and white photo of a fire hydrant a curb and fence
An old fire hydrant in the middle of the woods.
A bird is wading in the shallow water.
A made bed with mosquito netting above it.
A player that is getting ready to go up to bat at a baseball game.
A sign in front of a brick wall that says accident prone zone
Cat sat outside on a small folding wooden chair
a couple of sheep are in a grassy field
A bathroom with a toilet and standup shower.
Two teddy bears with balloons on a chair 
Fresh carrots on cabbage leaves on a formal place setting
a small wood table with a small computer on it 
A woman eating a plate of food at a table.
Two people in a bedroom holding wii remotes.
A group of people sit on the beach behind two surfboards.
A herd of zebras, ibex, and wilder beasts on the planes.
A meal, coffee, pictures and a notebook on a table.
A picture of various roadsigns and a crosswalk signal.
A man is standing in a train in a train station.
a couple of cellphones are laying out together
A person holding a sandwich made out of pretzels, cheese, and deli sliced meat.
A man and woman at a table with several drinks.
a zebra with his mouth open in a field.
An elephant is facing forward with zebras in the background. 
A white plate topped with meat veggies and rice with sauce.
A man is taking his picture in a mirror.
A kitchen has white cabinets and a white microwave.
A toaster is decorated with kitchen utensils to have a sad expression. 
Empty aqua bathroom with toilet, sink, mirror, and outside window.
a little girl is holding on to a hair brush
some glazed donuts are sitting on a white counter
A train carrying army trucks with men standing on it.
The two dogs are laying on the bed.
a close up of a painted parking meter on a street
A colorful market booth sells bananas to a customer.
Someone patting the head of a large stuffed teddy bear
Hang gliders fly over the water on a choppy day.
PEOPLE OBSERVING AND CLICKING PICTURES AT AN ANTIQUE PLACE.
A pizza with toppings sitting on a tray.
The center of a plant with lots of leaves.
An old man holding a plaque next to a motorcycle.
A cat that is sitting in a basket under a bench.
A cat sits in a blue patterned folding chair outdoors.
there are two elephants that are walking on the road
Old hot rods are sitting in a grass lot.
A man brushing his teeth, taking a picture of himself.
Cat looking out car window from car seat.
A double decker bus that is driving on the road.
A woman is sitting on an old wooden bench.
Two women playing a game with a Nintendo Wii controller.
A man on a motorcycle in heavy traffic 
A dog jumping through the air into a pool.
A group of people riding snow boards on top of a slope.
A skateboarder doing tricks on the edge of a wall.
Four way stop sign at street intersection and two street signs above
A cat sitting in a suitcase on a bed 
a man on a skateboard in the air coming on to a ramp
A view of a man wearing a casual shirt and tie.
A commercial international airline jet on a runway strip.
Two small birds are resting on a tree branch.
two people standing on a field with a kite flying above 
There are several kites flying over a crowd of people.
A small bird is perched on a branch in a tree.
A group of baseball players that are standing in the field.
A man standing on top of a tennis court.
Birthday cake for a friend that is ill
A group of giraffe walking across a dirt field.
A truck driving down a street next to a forest.
a large air plane flying in the air
an image of people posing for the camera
A boy smiles while two little dogs are inside of two tennis shoes.
A young man is taking a "selfie" of himself. 
A keyboard and some ear phones are on a table.
A large group of people eating under a tent.
Group of people riding on the back of large elephant. 
Long rows of white urinals sitting on a wall in a bathroom. 
A toilet with an apparatus to the side of it.
Working elephants in a village meander along a river bank. 
A plane equipped for water landing sits on a runway.
Three giraffe's laying in a grassy open area with trees. 
A group of horse back riders on a wooded trail.
A passenger jet airplane flying in a mostly cloudy sky.
A restaurant with patrons in the outdoor eating area.
A musician with a smile on his face prepares to play the violin.
A cheese omelet with toast on a plate.
A train car with purple and grey graffiti covering windows
A meat sandwich sitting on top of a white plate.
a bathroom with a mirror and a sink in it
A blue car passing a sign that reads "crepes ice cream gelato."
Several adults engaged in arts and crafts with children.
A set of bright yellow luggage with wheels.
A boy stares at a pizza with a birthday candle in it.
Boats in a river on a foggy day. 
A bathroom with three stalls and three different styles of urinals.
A motorcycle and couple of bicycles sitting on rugs in a store.
A plate of lamb chop, new potatoes, and steamed carrot slices.
A long sheet of pizza sitting on top of a table.
Large orange raft sits on boat deck attached to crane.
A bench under water in a park that has been flooded 
A boy standing in front of a tee ball tee with a bat.
As the sun sets, a boy and his father take their surfboard out of the water.
a big clock a rod iron roof and a round window
a train on railroad tracks at a station
Two women are skiing down a small hill.
A stop sign on the corner with garbage cans
Four donuts in a box with different toppings on them.
A very cute small girl by a very big pizza.
A green bird sitting on top of a desk chair.
An older man walking away holding a kite
A clock in the outdoors showing ten fifty. 
A cat laying on top of a blanket near a wall.
a dog with a plate of food on the ground
A tennis player preparing to hit the ball.
Colorful passenger trains make their way on parallel tracks.
Sheep in a field by the fence on a clear day.
some oranges are stacked up in a bowl
A puppy sitting on the ground in a bathroom
a man swinging a tennis racket on a tennis court
A  pair of people sitting on a motorcycle, in the grass.
an empty street and signals that are red
A wooden fishing boat in the middle of a grass field with snow capped mountains in the background.
A pair of men dressed in civil war outfits riding horses.
People with luggage are at the train station.
Two people playing a video game in their living room.
A very nice looking living and dining room fully furnished.
A man on skis going over a small jump.
An air mattress, plastic chair and bike decorate this room.
Several birds perched on a giraffe's neck eating bugs.
A group of people standing in the sand with a kite.
A boy on a skateboard hovers over a half pipe.
A long train travels past some trees along the railroad tracks
Two men dressed in business attire shaking hands.
A man goes through the water on a parasail.  
A man laying in bed with w child reading him a story
Two blankets have been folded on a large bed.
People looking at a large group of sheep in a fence in the middle of a city.
A plate of spaghetti with tomato sauce and a garnish of broccoli.
hotel room with two beds and a large window
a train that is on a old train track
A sailboat in the water, near a rocky island with a lighthouse on it.
Many bunches of fresh locally grown ripe bananas.
A bicycle is on the street in front of a window.
A batter up to bat at a baseball game.
A pizza sitting on top of a plate covered in cheese and tomatoes.
A group of men play indoor soccer together. 
a black and white photo of children siting posing for a photo
A blue teddy bear surrounded by other stuffed animals.
An apple computer with a keyboard and mouse underneath it.
A glass vase full of red flowers and greenery.
A man and women pose at a wedding.
The cardinal on the tail of a plane on the tarmac.
Two brown horses standing next to each other in a field.
Three buses pulled in front of a bus station loading passengers.
a person standing next to a snow board with mountains in the background
Four people are smiling and holding up smart phone devices.
a man on a skateboard jumping over a stairway
a kid is sitting in front of a laptop
A man and a boy standing next to a tall giraffe.
A smiling baby is laying in a suitcase.
a metal pole with a single sign is next to a wood pole with many signs
a yellow and blue fire hydrogen that is next to a road
A child tries to feed an adult a piece of food
a close up of a stuffed animal near a book
A black and white photo of a computer keyboard, flash drive, and mouse on a marble tabletop.
A woman holds onto a teddy bear while on a plane. 
A computer monitor sitting on top of a wooden table.
A kitchen that has carpeted floors and wooden cabinets.
A sign showing the way to Atlas Road.
A surfboard on top of a beaten car near the ocean
A group of people sitting next to each other in front of a TV.
a cat that is parked on a purple car
A plate with dinner items on it, on a white table cloth.
a few zebra are out in the open eating some hay
A man is going down the water on a paddle board.
The snow is very crowded with snow skiers.
A man posing behind a plate of food.
A stop sign leaning over in melting snow.
Man and woman sitting on a wooden park bench together. 
The twin boys are looking at the black bear.
a laptop with a mouse sitting on a towel 
A white dog is sitting on the back of a couch.
a furry brown hound sits on the snow in front of some garden patio furniture
A herd of zebra standing next to each other against a stone wall.
Bathroom sink with wooden cabinet and granite top.
A red stoplight with a street in the background.
A smiling man with a moustache is handed a plate by another man standing at a table with holiday decorations..
A man riding a bike down a street.
A little girl that is hitting a baseball.
A large group of people take a tour of a cargo plane.
Two men in blazers and fedoras with a moped in the background.
A woman with a black purse and a bath tub.
A woman taking a bite of a doughnut in a restaurant.
A group of steer with large horns lay in a field.
A park and walkway lined with benches and bushes.
It looks like this student is well-supplied with caffeine for an all-night cramming session.
An empty bathroom with a sink, toilet, scale and brown towel 
Woman throwing a rope towards horse at rodeo
A clean kitchen with warm sun light streaming in
A produce vendor and a customer on a sidewalk.
A group of people crossing a street next to a stop sign.
a city bus is parked along a deserted side street
some people standing on a tennis court holding tennis racketts
a snowboard instructor teaching students how to snowboard 
A white toilet sitting under a window in a bathroom.
a white plate with a variety of meat and vegetables
A pile of stuff on the floor next to a bed
A orange before and after it was cut
A uncooked whole chicken in a white microwave.
Two kids wearing skis and helmets in the snow.
a train moving on the rail near a forest
A woman walking with a pan in her hands with a whole pizza on it. 
A table full of many different types of food.
A jet airliner with landing gear down and flaps extended.
A man with a tie has his feet bent up in the air.  
A man is in front of an open oven door.
A muddy motorcycle is parked near other vehicles in a field.
A slice of pizza on a white plate.
Old cooking implements on wooden shelves in a white room.
PEOPLE CELEBRATING WITH CHOCOLATE CAKE AND WINE 
A purple rag laying in a toilet bowl.
Lilacs in a blue watering can vase sitting outdoors on a ledge.
A colorful bird sits on a tree limb.
a man standin on a sidewalk on a hill
A train car's open door exposes a sink and toilet room. 
an open read with trees lining each side of the road
A plant in a antique vase outside. 
Centuries old tower with a clock and topped with a cross.
the doll is a table with two stuffed dogs
Two people on a long rowboat in a river or lake.
a plate with three kinds of food on it
A street sign on a pole on a road.
a woman standing in a kitchen cooks some food
Three people on the beach standing with surfboards.
A car driving down a busy city street at night.
A golden elephant working as signage on a building facade.
A group of red and blue tourist buses going over a bridge.
Some boats parked in the water at a dock
a dog running with a Frisbee in its mouth
a surfer riding the wave at the beach
many people at tables with drinks and food 
A female tennis player is positioning herself for her next move.
The tray has a sandwich and two bowls near a beverage. 
here is a clock on the window that has too late written on it 
Giraffe standing in the shade of a tall covering 
a pole that has a bunch of street signs on it
A baseball game is being played before a crowd.
A group of zebras walk along the sand.
A horse has a cover over its mouth
Photo taking in a building looking at a staircase.
This studio apartment has a twin size bed by the wall.
A man cutting a cake with his bride.
A close up of a black and white teddy bear sitting on a bench.
a lady standing in front of potted plants.
Passengers wait at the platform as a passenger train approaches.
a man cutting a slice of cake with the knife
Five kids playing with frisbees on a sunny day.
Several birds stand and lay near the water.
A blue colored passenger bus that is parked on road next to a building.
A woman with a red tutu standing in the middle of a city street. 
A man with his focus looking up at the camera.
A herd of white sheep standing on a green hillside.
A road is covered with snow with some tire tracks.
There are tooth brush and gum like candies.
A cat sitting at the edge of a couch looking ahead.
A computer, lamp, and books sit on a desk.
A woman that is standing in the water next to a surfboard.
A bathroom with tall ceilings and an open window on the top.
a big train sits parked with graffiti on it 
A adult holding a child biting into a remote.
Two people look on as two others play a video game.
Professional baseball player sliding in to the base
Several dogs on a yellow school bus with a stop sign below the window.
A PICTURE OF BAR B QUE SITTING ON A TABLE 
A woman and a dog tussle over a frisbee.
A zebra and its foal eating hay from a basket.
a white purse sits on a bed under a phone, camera and flask
A YOUNG BLACK COW IS IN SOME SORT OF OUTDOOR INCLOSURE
A train parked in front of a train station.
An uneaten pizza pie cut into 4 quarters
A herd of horses gathered at the top of a hill.
Breakfast is complete with a large waffle topped with bananas.
A remote and cup held by a woman with glasses
a group of elephants near a body of water 
A couple of women sitting next to each other.
A photo of a reflection of a group of people lounging outside.
A man rides his wake board while paragliding
Young men riding skate boards along a marked path on the pavement.
black and white photo of a scooter carrying numerous bikes
A man standing in the ocean filled with waves.
A bedroom in the loft of a wooden cabin
a black and white dog a bicycle and a person and buildings
Two people having a meal at a diner.
A person in a deserted subway station waits for the train to pass. 
a white and blue jet is at the airport
3 surf boarders enjoying themselves in the ocean.
Woman adjusting man's tie in occupied room with others.
A work desk with a silver laptop propped up on top of the desk.
A woman preparing to hit a tennis ball while a man watches.
a person is holding up a small child
Barber cutting a child's hair in a barber shop.
two female sitting at a table across from eachother eating pizza
A man playing with a brown dog on top of a green field.
two small kids sitting at a table and on a lap top
a view into a kitchen with  counter tops and wooden floors
There are horses pulling a man and a cart.
Two women playing a video game in a living room.
A black and white cat sleeps next to a stuffed bear.
The baseball player is standing in the outfield. 
An air plane going down the runway on a clear day.
A whole pizza pie with many different toppings sitting on a pizza stand.
A facade to the front on an old church.
A meal at a restaurant of a salad, a toasted sandwich and a pickle

Writing for1k.txt


In [1]:
!python txt2img.py --from-file /kaggle/working/for1k.txt --ckpt /kaggle/input/stable-sig/stable_signature-main/v2-1_512-ema-pruned.ckpt --ldm_decoder_ckpt /kaggle/input/checkpointformine/checkpointnew_vgg.pth --config /kaggle/input/stable-sig/stable_signature-main/v2-inference.yaml  --H 512 --W 512 

python3: can't open file '/kaggle/working/txt2img.py': [Errno 2] No such file or directory


In [ ]:
%%writefile txt2img.py
import argparse, os
import cv2
import sys
sys.path.append('src')
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange
from torchvision.utils import make_grid
from pytorch_lightning import seed_everything
from torch import autocast
from contextlib import nullcontext
# from imwatermark import WatermarkEncoder
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from ldm.models.diffusion.dpm_solver import DPMSolverSampler

torch.set_grad_enabled(False)

def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

def insert_watermark_in_latents(samples, watermark_latent, alpha=0.05):
    """
    Inserts an imperceptible watermark into the latent representation.

    Args:
        samples (torch.Tensor): The latent representation of the image (B, C, H, W).
        watermark_latent (DiagonalGaussianDistribution): The latent representation of the watermark.
        alpha (float): Scaling factor for the watermark (default: 0.1, adjust as needed).

    Returns:
        torch.Tensor: The modified latent representation with the imperceptible watermark.
    """
    # Ensure watermark_latent is sampled correctly
    if hasattr(watermark_latent, 'sample'):
        watermark_latent_tensor = watermark_latent.sample()  # Call sample as a method
    else:
        raise TypeError(f"Expected a DiagonalGaussianDistribution, got {type(watermark_latent)}")

    # Get the shape of the watermark latent tensor
    wm_h, wm_w = watermark_latent_tensor.shape[2], watermark_latent_tensor.shape[3]

    # Compute the center of the image latent tensor
    center_h, center_w = samples.shape[2] // 2, samples.shape[3] // 2

    # Determine the position to insert the watermark
    start_h, start_w = center_h - wm_h // 2, center_w - wm_w // 2

    # Blend the watermark latents into the image latents
    samples[:, :, start_h:start_h+wm_h, start_w:start_w+wm_w] += alpha * watermark_latent_tensor

    return samples


def load_model_from_config(config, ckpt, device=torch.device("cuda"), verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    if device == torch.device("cuda"):
        model.cuda()
    elif device == torch.device("cpu"):
        model.cpu()
        model.cond_stage_model.device = "cpu"
    else:
        raise ValueError(f"Incorrect device name. Received: {device}")
    model.eval()
    return model

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--prompt",
        type=str,
        nargs="?",
        default="a professional photograph of an astronaut riding a triceratops",
        help="the prompt to render"
    )
    parser.add_argument(
        "--outdir",
        type=str,
        nargs="?",
        help="dir to write results to",
        default="outputs/txt2img-samples"
    )
    parser.add_argument(
        "--steps",
        type=int,
        default=50,
        help="number of ddim sampling steps",
    )
    parser.add_argument(
        "--plms",
        action='store_true',
        help="use plms sampling",
    )
    parser.add_argument(
        "--dpm",
        action='store_true',
        help="use DPM (2) sampler",
    )
    parser.add_argument(
        "--fixed_code",
        action='store_true',
        help="if enabled, uses the same starting code across all samples ",
    )
    parser.add_argument(
        "--ddim_eta",
        type=float,
        default=0.0,
        help="ddim eta (eta=0.0 corresponds to deterministic sampling",
    )
    parser.add_argument(
        "--n_iter",
        type=int,
        default=1,
        help="sample this often",
    )
    parser.add_argument(
        "--H",
        type=int,
        default=512,
        help="image height, in pixel space",
    )
    parser.add_argument(
        "--W",
        type=int,
        default=512,
        help="image width, in pixel space",
    )
    parser.add_argument(
        "--C",
        type=int,
        default=4,
        help="latent channels",
    )
    parser.add_argument(
        "--f",
        type=int,
        default=8,
        help="downsampling factor, most often 8 or 16",
    )
    parser.add_argument(
        "--n_samples",
        type=int,
        default=1,
        help="how many samples to produce for each given prompt. A.k.a batch size",
    )
    parser.add_argument(
        "--n_rows",
        type=int,
        default=0,
        help="rows in the grid (default: n_samples)",
    )
    parser.add_argument(
        "--ldm_decoder_ckpt",
        type=str,
        default=None,
        # default="/kaggle/input/forstatistics/checkpoint_009.pth",
    )
    parser.add_argument(
        "--scale",
        type=float,
        default=3.0,
        help="unconditional guidance scale: eps = eps(x, empty) + scale * (eps(x, cond) - eps(x, empty))",
    )
    parser.add_argument(
        "--from-file",
        type=str,
        help="if specified, load prompts from this file, separated by newlines",
    )
    parser.add_argument(
        "--config",
        type=str,
        default="configs/stable-diffusion/v2-inference.yaml",
        help="path to config which constructs model",
    )
    parser.add_argument(
        "--ckpt",
        type=str,
        help="path to checkpoint of model",
    )
    parser.add_argument(
        "--seed",
        type=int,
        default=42,
        help="the seed (for reproducible sampling)",
    )
    parser.add_argument(
        "--precision",
        type=str,
        help="evaluate at this precision",
        choices=["full", "autocast"],
        default="autocast"
    )
    parser.add_argument(
        "--repeat",
        type=int,
        default=1,
        help="repeat each prompt in file this often",
    )
    parser.add_argument(
        "--device",
        type=str,
        help="Device on which Stable Diffusion will be run",
        choices=["cpu", "cuda"],
        default="cuda"
    )
    parser.add_argument(
        "--torchscript",
        action='store_true',
        help="Use TorchScript",
    )
    parser.add_argument(
        "--ipex",
        action='store_true',
        help="Use Intel® Extension for PyTorch*",
    )
    # parser.add_argument(
    #     "--bf16",
    #     action='store_true',
    #     help="Use bfloat16",
    # )
    opt = parser.parse_args()
    return opt

def main(opt):
    seed_everything(opt.seed)

    config = OmegaConf.load(f"{opt.config}")
    device = torch.device("cuda") if opt.device == "cuda" else torch.device("cpu")
    model = load_model_from_config(config, f"{opt.ckpt}", device)
    # Parameter None for clutil sweep
    print(f'reload decoder weights {opt.ldm_decoder_ckpt}...')
    if opt.ldm_decoder_ckpt is not None and opt.ldm_decoder_ckpt.lower() == "none":
        opt.ldm_decoder_ckpt = None
    if opt.ldm_decoder_ckpt is not None:
        state_dict = torch.load(opt.ldm_decoder_ckpt)['ldm_decoder']
        # state_dict = torch.load(opt.ldm_decoder_ckpt)['state_dict']
        # state_dict = {k.replace('first_stage_model.', ''): v for k, v in state_dict.items() if 'decoder' in k or 'post_quant_conv' in k}
        msg = model.first_stage_model.load_state_dict(state_dict, strict=False)
        model.eval()
        print(msg)

    if opt.plms:
        sampler = PLMSSampler(model, device=device)
    elif opt.dpm:
        sampler = DPMSolverSampler(model, device=device)
    else:
        sampler = DDIMSampler(model, device=device)

    os.makedirs(opt.outdir, exist_ok=True)
    outpath = opt.outdir
    batch_size = opt.n_samples
    n_rows = opt.n_rows if opt.n_rows > 0 else batch_size
    if not opt.from_file:
        prompt = opt.prompt
        assert prompt is not None
        data = [batch_size * [prompt]]
    else:
        print(f"reading prompts from {opt.from_file}")
        with open(opt.from_file, "r") as f:
            data = f.read().splitlines()
            data = [p for p in data for i in range(opt.repeat)]
            data = list(chunk(data, batch_size))

    sample_path = os.path.join(outpath, "samplesnonwater50k")
    os.makedirs(sample_path, exist_ok=True)
    sample_count = 0
    base_count = len(os.listdir(sample_path))
    grid_count = len(os.listdir(outpath)) - 1

    start_code = None
    if opt.fixed_code:
        start_code = torch.randn([opt.n_samples, opt.C, opt.H // opt.f, opt.W // opt.f], device=device)
        
    if opt.torchscript or opt.ipex:
        transformer = model.cond_stage_model.model
        unet = model.model.diffusion_model
        decoder = model.first_stage_model.decoder
        additional_context = torch.cpu.amp.autocast() if opt.bf16 else nullcontext()
        shape = [opt.C, opt.H // opt.f, opt.W // opt.f]

        if opt.bf16 and not opt.torchscript and not opt.ipex:
            raise ValueError('Bfloat16 is supported only for torchscript+ipex')
        if opt.bf16 and unet.dtype != torch.bfloat16:
            raise ValueError("Use configs/stable-diffusion/intel/ configs with bf16 enabled if " +
                             "you'd like to use bfloat16 with CPU.")
        if unet.dtype == torch.float16 and device == torch.device("cpu"):
            raise ValueError("Use configs/stable-diffusion/intel/ configs for your model if you'd like to run it on CPU.")

        if opt.ipex:
            import intel_extension_for_pytorch as ipex
            bf16_dtype = torch.bfloat16 if opt.bf16 else None
            transformer = transformer.to(memory_format=torch.channels_last)
            transformer = ipex.optimize(transformer, level="O1", inplace=True)

            unet = unet.to(memory_format=torch.channels_last)
            unet = ipex.optimize(unet, level="O1", auto_kernel_selection=True, inplace=True, dtype=bf16_dtype)

            decoder = decoder.to(memory_format=torch.channels_last)
            decoder = ipex.optimize(decoder, level="O1", auto_kernel_selection=True, inplace=True, dtype=bf16_dtype)

        if opt.torchscript:
            with torch.no_grad(), additional_context:
                # get UNET scripted
                if unet.use_checkpoint:
                    raise ValueError("Gradient checkpoint won't work with tracing. " +
                    "Use configs/stable-diffusion/intel/ configs for your model or disable checkpoint in your config.")

                img_in = torch.ones(2, 4, 96, 96, dtype=torch.float32)
                t_in = torch.ones(2, dtype=torch.int64)
                context = torch.ones(2, 77, 1024, dtype=torch.float32)
                scripted_unet = torch.jit.trace(unet, (img_in, t_in, context))
                scripted_unet = torch.jit.optimize_for_inference(scripted_unet)
                print(type(scripted_unet))
                model.model.scripted_diffusion_model = scripted_unet

                # get Decoder for first stage model scripted
                samples_ddim = torch.ones(1, 4, 96, 96, dtype=torch.float32)
                scripted_decoder = torch.jit.trace(decoder, (samples_ddim))
                scripted_decoder = torch.jit.optimize_for_inference(scripted_decoder)
                print(type(scripted_decoder))
                model.first_stage_model.decoder = scripted_decoder

        prompts = data[0]
        print("Running a forward pass to initialize optimizations")
        uc = None
        if opt.scale != 1.0:
            uc = model.get_learned_conditioning(batch_size * [""])
        if isinstance(prompts, tuple):
            prompts = list(prompts)

        with torch.no_grad(), additional_context:
            for _ in range(3):
                c = model.get_learned_conditioning(prompts)
            samples_ddim, _ = sampler.sample(S=5,
                                             conditioning=c,
                                             batch_size=batch_size,
                                             shape=shape,
                                             verbose=False,
                                             unconditional_guidance_scale=opt.scale,
                                             unconditional_conditioning=uc,
                                             eta=opt.ddim_eta,
                                             x_T=start_code)
            print("Running a forward pass for decoder")
            for _ in range(3):
                x_samples_ddim = model.decode_first_stage(samples_ddim)

    precision_scope = autocast if opt.precision=="autocast" or opt.bf16 else nullcontext
    with torch.no_grad(), \
        precision_scope(opt.device), \
        model.ema_scope():
            all_samples = list()
            for n in trange(opt.n_iter, desc="Sampling"):
                for prompts in tqdm(data, desc="data"):
                    uc = None
                    if opt.scale != 1.0:
                        uc = model.get_learned_conditioning(batch_size * [""])
                    if isinstance(prompts, tuple):
                        prompts = list(prompts)
                    c = model.get_learned_conditioning(prompts)
                    shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                    samples, _ = sampler.sample(S=opt.steps,
                                                     conditioning=c,
                                                     batch_size=opt.n_samples,
                                                     shape=shape,
                                                     verbose=False,
                                                     unconditional_guidance_scale=opt.scale,
                                                     unconditional_conditioning=uc,
                                                     eta=opt.ddim_eta,
                                                     x_T=start_code)
                    watermark_image = Image.open('/kaggle/input/mnistasjpg/testSet/testSet/img_100.jpg').convert('RGB')  # assuming RGB for color watermark
                    watermark_image = watermark_image.resize((32, 32), Image.Resampling.LANCZOS)
                    watermark_tensor = torch.tensor(np.array(watermark_image), dtype=torch.float32)  # Normalize to [0, 1]
                    watermark_tensor = watermark_tensor.permute(2, 0, 1).unsqueeze(0)  # Add batch dim and change channel order to CxHxW

            # Pass the watermark through the encoder to get the latent representation
                    watermark_latent = model.encode_first_stage(watermark_tensor.to(opt.device))  # Assuming an encoder method exists
            # Insert the watermark latent into the samples
                    modified_latents = insert_watermark_in_latents(samples, watermark_latent)
                    # x_samples = model.decode_first_stage(samples)
                    if opt.ldm_decoder_ckpt is not None and opt.ldm_decoder_ckpt.lower() == "none":
                        x_samples = model.decode_first_stage(samples)
                    else:
                        x_samples = model.decode_first_stage(modified_latents)
                    x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                    for x_sample in x_samples:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                        img = Image.fromarray(x_sample.astype(np.uint8))
                        # img = put_watermark(img, wm_encoder)
                        img.save(os.path.join(sample_path, f"{base_count:05}.png"))
                        base_count += 1
                        sample_count += 1

                    all_samples.append(x_samples)

            # # additionally, save as grid
            # grid = torch.stack(all_samples, 0)
            # grid = rearrange(grid, 'n b c h w -> (n b) c h w')
            # grid = make_grid(grid, nrow=n_rows)

            # # to image
            # grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
            # grid = Image.fromarray(grid.astype(np.uint8))
            # # grid = put_watermark(grid, wm_encoder)
            # grid.save(os.path.join(outpath, f'grid-{grid_count:04}.png'))
            # grid_count += 1

    print(f"Your samples are ready and waiting for you here: \n{outpath} \n"
          f" \nEnjoy.")

if __name__ == "__main__":
    opt = parse_args()
    main(opt)

In [ ]:
!python txt2img.py --from-file /kaggle/working/for1k.txt --ckpt /kaggle/input/stable-sig/stable_signature-main/v2-1_512-ema-pruned.ckpt --ldm_decoder_ckpt none --config /kaggle/input/stable-sig/stable_signature-main/v2-inference.yaml  --H 512 --W 512 

In [ ]:
!zip -r first1kk.zip /kaggle/working/outputs